In [1]:
%matplotlib inline
### Standard imports
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from importlib import reload  

### HEPKIT imports
import hepkit.hepData as hd
import hepkit.hepPlot as hp
import hepkit.hepFormat as hf
### Settings
hf.DefaultSetup()
hf.DefaultSetup()

# Necessary imports
import support as sp
from googletrans import Translator
import spacy
import time

In [20]:
# Settings
dirIn = '/Users/sdporzio/Software/ebook_trans/input'
dirOut = '/Users/sdporzio/Software/ebook_trans/output'
filename = 'alquimista'

nlp_model = 'pt_core_news_md'
src, dest = 'pt', 'en'

maxCharacters = 300

In [21]:
reload(sp)
# Load model
nlp = spacy.load(nlp_model)
# Load google translator
translator = Translator()

fin = dirIn+'/'+filename+'.txt'
fout = dirOut+'/'+filename+'.txt'

# Load text
with open(fin,'rb') as fileIn:
    text = fileIn.read()
# Decode it
text = text.decode(encoding='utf8')
# Remove stupid empty lines and split in paragraphs
text = sp.RemoveLines(text)
pars = text.split('\n\n')
# Split long paragraphs in shorter ones
new_pars = []
for p in tqdm(pars):
    shorter_p = sp.RebinPhrases(nlp,p,maxCharacters)
    for shp in shorter_p:
        if shp!='': new_pars.append(shp)

In [22]:
reload(sp)
# Compose text with translation
out_text = ''
lims = sp.GetDivisions(new_pars,15)
for lim in tqdm(lims):
    start = lim[0]
    end = lim[1]
    new_block = new_pars[start:end]
    new_merged_block = '\n'.join(new_block)
    new_tblock = new_merged_block
    attempts = 0
    while (new_tblock == new_merged_block and attempts<20) :
        new_tblock = translator.translate(new_merged_block,src=src,dest=dest).text
        time.sleep(0.2)
        attempts+=1
    if attempts>=20:
        print("Gave up at:")
        print(new_merged_block)
    split_tblock = new_tblock.split('\n')
    for i in range(len(new_block)):
        out_text += new_block[i]
        out_text += '\n\n'
        out_text += split_tblock[i]
        out_text += '\n\n\n'
        
# Save out to text
with open(fout,'w') as fileOut:
    text = fileOut.write(out_text)
    
hd.SlackMe("Ebook converted.")